# Evaluationsmethoden für LLM-Anwendungen

In diesem Notebook lernen Sie verschiedene Methoden zur Evaluation von LLM-Anwendungen kennen. Wir werden folgende Ansätze untersuchen:

- LLM-As-A-Judge: Bewertung von LLM-Antworten durch andere LLMs
- NLP-basierte Testkriterien (ROUGE, BLEU)
- PI Scrubbing und Datenschutz mit Microsoft Presidio
- RAG-spezifische Evaluierungsmethoden

Diese Techniken helfen Ihnen, die Qualität Ihrer KI-Anwendungen systematisch zu verbessern.

## 1. Installation und Konfiguration der benötigten Bibliotheken

In [ ]:
# Installation der erforderlichen Pakete
!pip install langchain langchain_openai langchain-core rouge-score nltk presidio-analyzer presidio-anonymizer -q

# Umgebungsvariablen laden
import os
from dotenv import load_dotenv
load_dotenv()

# Grundlegende Importe
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# LangChain Importe
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.evaluation import load_evaluator
from langchain.evaluation.criteria import LabeledCriteriaEvaluator
from langchain_core.messages import HumanMessage, SystemMessage

In [ ]:
# Initialisieren des LLMs
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Für Bewertungen verwenden wir ein separates LLM-Objekt
evaluator_llm = ChatOpenAI(model="gpt-4o", temperature=0)

## 2. LLM-As-A-Judge: Grundlagen und Implementierung

LLM-As-A-Judge ist ein Ansatz, bei dem ein LLM zur Bewertung der Ausgaben eines anderen LLMs verwendet wird. Dies ist besonders nützlich, wenn keine Referenzantworten verfügbar sind oder die Bewertung menschlicher Präferenzen simuliert werden soll.

In [ ]:
# Beispielantworten für die Bewertung
example_question = "Was sind die wichtigsten Anwendungsfälle für Vektordatenbanken im Kontext von RAG-Systemen?"

response_a = """Vektordatenbanken werden in RAG-Systemen hauptsächlich für die effiziente Speicherung und den Abruf von Embeddings verwendet. 
Sie ermöglichen semantische Suche basierend auf Ähnlichkeit statt einfache Keyword-Suche. 
Weitere Anwendungsfälle sind Clusteranalyse und die Reduzierung der Dimensionalität großer Datensätze."""

response_b = """Vektordatenbanken speichern Daten. Sie werden mit RAG verwendet, um Texte zu durchsuchen. 
Man kann darin Daten speichern und wieder abrufen, wenn man sie braucht. 
Sie sind besser als normale Datenbanken, weil sie schneller sind."""

In [ ]:
# 1. Evaluierung mit einem vordefinierten Evaluator
qa_evaluator = load_evaluator("labeled_criteria", criteria={
    "relevanz": "Ist die Antwort relevant für die Frage?",
    "korrektheit": "Enthält die Antwort korrekte und aktuelle Informationen?",
    "vollständigkeit": "Beantwortet die Antwort die Frage vollständig?",
    "klarheit": "Ist die Antwort klar, präzise und gut strukturiert?"
}, llm=evaluator_llm)

# Bewertung der Antworten
eval_result_a = qa_evaluator.evaluate_strings(
    prediction=response_a,
    input=example_question
)

eval_result_b = qa_evaluator.evaluate_strings(
    prediction=response_b,
    input=example_question
)

print("Bewertung Antwort A:")
for criterion, rating in eval_result_a["criteria"].items():
    print(f"{criterion}: {rating['rating']} - {rating['reasoning']}")
    
print("\nBewertung Antwort B:")
for criterion, rating in eval_result_b["criteria"].items():
    print(f"{criterion}: {rating['rating']} - {rating['reasoning']}")

### Eigenen LLM-As-A-Judge Evaluator implementieren

LangChain bietet vordefinierte Evaluatoren, aber manchmal benötigen wir spezifischere Bewertungskriterien. Hier erstellen wir einen eigenen Evaluator.

In [ ]:
# Eigenen Evaluator mit spezifischen Kriterien erstellen
custom_criteria = {
    "technische_genauigkeit": "Enthält die Antwort technisch korrekte Informationen über Vektordatenbanken und RAG?",
    "praxisbezug": "Gibt die Antwort praktische Beispiele oder Anwendungsfälle?",
    "zielgruppenorientierung": "Ist die Antwort für Entwickler verständlich und hilfreich?",
    "vollständigkeit": "Deckt die Antwort alle wichtigen Aspekte von Vektordatenbanken im RAG-Kontext ab?"
}

custom_evaluator = LabeledCriteriaEvaluator(criteria=custom_criteria, llm=evaluator_llm)

# Bewertung mit dem benutzerdefinierten Evaluator
custom_eval_a = custom_evaluator.evaluate_strings(
    prediction=response_a,
    input=example_question
)

print("Benutzerdefinierte Bewertung für Antwort A:")
for criterion, rating in custom_eval_a["criteria"].items():
    print(f"{criterion}: {rating['rating']} - {rating['reasoning']}")

### Vergleichende Bewertung

Manchmal ist es sinnvoller, Antworten direkt miteinander zu vergleichen, anstatt sie einzeln zu bewerten.

In [ ]:
# Vergleichenden Evaluator laden
pairwise_evaluator = load_evaluator("pairwise_string")

# Direkte Gegenüberstellung der Antworten
comparison = pairwise_evaluator.evaluate_string_pairs(
    prediction_a=response_a,
    prediction_b=response_b,
    input=example_question
)

print(f"Bevorzugte Antwort: {comparison['preferred']}")
print(f"Begründung: {comparison['reasoning']}")

## 3. NLP-basierte Testkriterien (ROUGE, BLEU, etc.)

Neben LLM-basierten Evaluierungsmethoden können wir auch traditionelle NLP-Metriken verwenden, die auf Textähnlichkeit basieren. Diese sind besonders nützlich, wenn wir Referenzantworten haben.

In [ ]:
# ROUGE-Metrik importieren
from rouge_score import rouge_scorer

# Referenzantwort (Gold-Standard)
reference_answer = """Vektordatenbanken spielen eine entscheidende Rolle in RAG-Systemen und bieten folgende Hauptanwendungsfälle:
1. Effiziente Speicherung und Indexierung von Embeddings für schnellen Abruf
2. Semantische Suche basierend auf Vektorähnlichkeit statt nur Schlüsselwörtern
3. Filterung und Abfrage mit Metadaten zusätzlich zur Vektorsuche
4. Skalierbare Verwaltung großer Dokumentenkorpora für RAG-Anwendungen
5. Unterstützung für komplexe Abfragen mit Hybrid-Retrieval-Strategien"""

# ROUGE-Scorer initialisieren
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# ROUGE-Scores berechnen
rouge_scores_a = scorer.score(reference_answer, response_a)
rouge_scores_b = scorer.score(reference_answer, response_b)

print("ROUGE-Scores für Antwort A:")
for metric, score in rouge_scores_a.items():
    print(f"{metric}: Precision = {score.precision:.4f}, Recall = {score.recall:.4f}, F1 = {score.fmeasure:.4f}")

print("\nROUGE-Scores für Antwort B:")
for metric, score in rouge_scores_b.items():
    print(f"{metric}: Precision = {score.precision:.4f}, Recall = {score.recall:.4f}, F1 = {score.fmeasure:.4f}")

In [ ]:
# BLEU-Score berechnen
import nltk
from nltk.translate.bleu_score import sentence_bleu

# NLTK-Daten herunterladen, falls noch nicht erfolgt
nltk.download('punkt', quiet=True)

# Tokenisieren der Texte
reference_tokens = nltk.word_tokenize(reference_answer.lower())
response_a_tokens = nltk.word_tokenize(response_a.lower())
response_b_tokens = nltk.word_tokenize(response_b.lower())

# BLEU-Scores berechnen
bleu_a = sentence_bleu([reference_tokens], response_a_tokens)
bleu_b = sentence_bleu([reference_tokens], response_b_tokens)

print(f"BLEU-Score für Antwort A: {bleu_a:.4f}")
print(f"BLEU-Score für Antwort B: {bleu_b:.4f}")

### Visualisierung der Ergebnisse

In [ ]:
# Ergebnisse visualisieren
metrics = ['ROUGE-1 F1', 'ROUGE-2 F1', 'ROUGE-L F1', 'BLEU']
scores_a = [rouge_scores_a['rouge1'].fmeasure, rouge_scores_a['rouge2'].fmeasure, 
            rouge_scores_a['rougeL'].fmeasure, bleu_a]
scores_b = [rouge_scores_b['rouge1'].fmeasure, rouge_scores_b['rouge2'].fmeasure, 
            rouge_scores_b['rougeL'].fmeasure, bleu_b]

x = np.arange(len(metrics))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(x - width/2, scores_a, width, label='Antwort A')
rects2 = ax.bar(x + width/2, scores_b, width, label='Antwort B')

ax.set_ylabel('Scores')
ax.set_title('Vergleich der Evaluationsmetriken')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()

fig.tight_layout()
plt.show()

## 4. PI Scrubbing und Datenschutz mit Microsoft Presidio

Bei der Evaluierung von LLM-Anwendungen ist Datenschutz ein wichtiger Aspekt. Microsoft Presidio ist ein Open-Source-Projekt für die Erkennung und Anonymisierung personenbezogener Daten (PII).

In [ ]:
# Presidio-Komponenten importieren
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine

# Analyzer und Anonymizer initialisieren
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

In [ ]:
# Beispieltext mit personenbezogenen Daten
text_mit_pii = """
Sehr geehrter Herr Schmidt,

vielen Dank für Ihre Anfrage. Bitte kontaktieren Sie mich unter meiner E-Mail max.mustermann@example.com oder telefonisch unter +49 176 12345678.

Ihr Kundenkonto mit der IBAN DE89 3704 0044 0532 0130 00 wurde aktualisiert.

Mit freundlichen Grüßen,
Dr. Anna Weber
"""

# PII erkennen (deutsche Sprache)
analyzer_results = analyzer.analyze(text=text_mit_pii, language="de")

print("Erkannte personenbezogene Daten:")
for result in analyzer_results:
    print(f"- {result.entity_type}: '{text_mit_pii[result.start:result.end]}'")

In [ ]:
# Text anonymisieren
anonymized_text = anonymizer.anonymize(
    text=text_mit_pii,
    analyzer_results=analyzer_results
)

print("Anonymisierter Text:")
print(anonymized_text.text)

In [ ]:
# Funktion zum Anonymisieren von Evaluationsdaten
def prepare_safe_evaluation_data(raw_text):
    # PII erkennen
    results = analyzer.analyze(text=raw_text, language="de")
    
    # Wenn PII gefunden wurde, anonymisieren
    if results:
        anonymized = anonymizer.anonymize(
            text=raw_text,
            analyzer_results=results
        )
        return anonymized.text, True
    else:
        return raw_text, False

# Beispiel für eine sichere Evaluierung
evaluation_text = """Herr Peter Müller (geboren am 15.04.1975) hat eine Anfrage zu seinem Konto 98765432 geschickt.
Seine Telefonnummer lautet 030 12345678. Bitte evaluieren Sie die Qualität der Antwort."""

safe_text, was_anonymized = prepare_safe_evaluation_data(evaluation_text)
print(f"Text wurde anonymisiert: {was_anonymized}")
print(f"Sicherer Text für Evaluierung:\n{safe_text}")

## 5. RAG-spezifische Evaluierungsmethoden

Für RAG-Systeme (Retrieval Augmented Generation) gibt es spezifische Evaluierungsmethoden, die sowohl die Retrieval-Komponente als auch die Generierungskomponente bewerten.

In [ ]:
# Beispiel für RAG-Evaluierung
from langchain.prompts import PromptTemplate

# Einfacher RAG-Prozess simulieren
def simulate_rag(query, documents):
    # Dokumente als Kontext zusammenfügen
    context = "\n\n".join([f"Dokument {i+1}: {doc}" for i, doc in enumerate(documents)])
    
    # Prompt für die Generierung
    prompt_template = PromptTemplate.from_template(
        """Du bist ein hilfreicher Assistent. Verwende den folgenden Kontext, um die Frage zu beantworten.
        
        Kontext:
        {context}
        
        Frage: {query}
        
        Deine Antwort:"""
    )
    
    # Chain erstellen und ausführen
    chain = LLMChain(llm=llm, prompt=prompt_template)
    result = chain.invoke({"context": context, "query": query})
    
    return result["text"], context

In [ ]:
# Beispiel-Dokumente
rag_documents = [
    "Vektordatenbanken sind spezielle Datenbanken, die für die Speicherung und Abfrage von Vektoren (Embeddings) optimiert sind. Sie ermöglichen schnelle Ähnlichkeitssuchen in hochdimensionalen Räumen.",
    "RAG (Retrieval Augmented Generation) ist ein Ansatz, bei dem ein LLM mit externen Informationen angereichert wird. Dadurch wird die Faktengenauigkeit verbessert und Halluzinationen werden reduziert.",
    "Die Hauptvorteile von Vektordatenbanken in RAG-Systemen sind: schnelle Ähnlichkeitssuche, skalierbare Speicherung von Embeddings und die Möglichkeit zur Metadatenfilterung.",
    "Chroma und Qdrant sind beliebte Vektordatenbanken für RAG-Anwendungen. Chroma ist einfach zu nutzen und ideal für Prototyping, während Qdrant für Produktionsanwendungen mit hohen Anforderungen geeignet ist."
]

# RAG-Anfrage und Antwort generieren
rag_query = "Welche Vektordatenbanken sind für RAG-Systeme am besten geeignet und warum?"
rag_answer, rag_context = simulate_rag(rag_query, rag_documents)

print("RAG-Antwort:")
print(rag_answer)

In [ ]:
# RAG-spezifische Evaluierung

# 1. Kontext-Relevanz prüfen
context_relevance_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="Du bist ein Experte für die Bewertung von Retrieval-Systemen. Deine Aufgabe ist es, die Relevanz der zurückgegebenen Dokumente für eine Anfrage zu bewerten."),
    HumanMessage(content="""Bitte bewerte die Relevanz der folgenden Dokumente für die gegebene Anfrage. 
    Gib jedem Dokument eine Bewertung von 1-5 (1 = irrelevant, 5 = hochrelevant) und eine kurze Begründung.
    
    Anfrage: {query}
    
    Zurückgegebene Dokumente:
    {context}
    """)
])

context_eval_chain = LLMChain(llm=evaluator_llm, prompt=context_relevance_prompt)
context_eval = context_eval_chain.invoke({"query": rag_query, "context": rag_context})

print("Bewertung der Kontext-Relevanz:")
print(context_eval["text"])

In [ ]:
# 2. Treue zum Kontext (Faithfulness) prüfen
faithfulness_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="Du bist ein Experte für die Bewertung von KI-generierten Antworten. Deine Aufgabe ist es, zu prüfen, ob eine Antwort durch den gegebenen Kontext unterstützt wird."),
    HumanMessage(content="""Bitte analysiere, ob die folgende Antwort vollständig durch den gegebenen Kontext unterstützt wird. 
    Identifiziere alle Behauptungen in der Antwort und bewerte, ob sie im Kontext enthalten sind oder Halluzinationen darstellen.
    
    Kontext:
    {context}
    
    Antwort:
    {answer}
    
    Gib eine Gesamtbewertung der Treue zum Kontext (1-5, wobei 5 die höchste Treue bedeutet) und liste alle Halluzinationen oder nicht unterstützten Aussagen auf.
    """)
])

faithfulness_chain = LLMChain(llm=evaluator_llm, prompt=faithfulness_prompt)
faithfulness_eval = faithfulness_chain.invoke({"context": rag_context, "answer": rag_answer})

print("Bewertung der Treue zum Kontext:")
print(faithfulness_eval["text"])

In [ ]:
# 3. Antwortqualität bewerten
quality_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="Du bist ein Experte für die Bewertung von KI-generierten Antworten. Deine Aufgabe ist es, die Qualität einer Antwort zu bewerten."),
    HumanMessage(content="""Bitte bewerte die Qualität der folgenden Antwort auf die gegebene Frage nach diesen Kriterien:
    1. Vollständigkeit (1-5): Beantwortet die Antwort alle Aspekte der Frage?
    2. Korrektheit (1-5): Enthält die Antwort korrekte Informationen?
    3. Klarheit (1-5): Ist die Antwort klar und verständlich formuliert?
    4. Prägnanz (1-5): Ist die Antwort präzise und auf den Punkt, ohne unnötige Informationen?
    
    Frage: {query}
    
    Antwort:
    {answer}
    
    Gib für jedes Kriterium eine Bewertung und eine kurze Begründung. Berechne auch einen Gesamtscore als Durchschnitt aller Kriterien.
    """)
])

quality_chain = LLMChain(llm=evaluator_llm, prompt=quality_prompt)
quality_eval = quality_chain.invoke({"query": rag_query, "answer": rag_answer})

print("Bewertung der Antwortqualität:")
print(quality_eval["text"])


## 6. Ganzheitliche RAG-Evaluierung: RAGAS

RAGAS ist ein Framework zur Evaluierung von RAG-Anwendungen, das verschiedene Aspekte eines RAG-Systems bewertet. Hier zeigen wir eine vereinfachte Version der RAGAS-Metriken.

In [ ]:
# Vereinfachte RAGAS-Evaluierung
ragas_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="Du bist ein Experte für die Evaluierung von RAG-Systemen (Retrieval Augmented Generation). Deine Aufgabe ist es, ein RAG-System nach den RAGAS-Metriken zu bewerten."),
    HumanMessage(content="""Bitte evaluiere das folgende RAG-System nach diesen RAGAS-Metriken:
    
    1. Kontext-Relevanz: Sind die abgerufenen Dokumente relevant für die Anfrage?
    2. Treue (Faithfulness): Wird die Antwort durch den Kontext unterstützt?
    3. Antwort-Relevanz: Beantwortet die generierte Antwort die ursprüngliche Frage?
    4. Kontext-Nutzung: Wie gut nutzt die Antwort den bereitgestellten Kontext?
    
    Anfrage: {query}
    
    Abgerufene Dokumente:
    {context}
    
    Generierte Antwort:
    {answer}
    
    Gib für jede Metrik eine Bewertung von 0-1 (0 = schlecht, 1 = hervorragend) und berechne einen Gesamtscore als gewichteten Durchschnitt.
    Gib außerdem konkrete Verbesserungsvorschläge für das RAG-System.
    """)
])

ragas_chain = LLMChain(llm=evaluator_llm, prompt=ragas_prompt)
ragas_eval = ragas_chain.invoke({"query": rag_query, "context": rag_context, "answer": rag_answer})

print("RAGAS-Evaluierung:")
print(ragas_eval["text"])

## 7. Systematische Evaluierung mit einem Testdatensatz

Für eine gründliche Evaluierung sollten wir mehrere Anfragen testen und die Ergebnisse aggregieren.

In [ ]:
# Testdatensatz mit Fragen und erwarteten Antworten
test_dataset = [
    {
        "query": "Welche Vektordatenbanken sind für RAG-Systeme am besten geeignet?",
        "expected_answer": "Chroma eignet sich gut für Prototyping, während Qdrant für Produktionsanwendungen empfohlen wird."
    },
    {
        "query": "Was ist der Hauptvorteil von Vektordatenbanken in RAG?",
        "expected_answer": "Schnelle Ähnlichkeitssuche und skalierbare Speicherung von Embeddings."
    },
    {
        "query": "Erkläre den Unterschied zwischen RAG und reinen LLM-Anwendungen.",
        "expected_answer": "RAG ergänzt LLMs mit externen Informationen zur Verbesserung der Faktengenauigkeit."
    }
]

# Funktion zur Durchführung einer systematischen Evaluierung
def evaluate_rag_system(test_cases, documents):
    results = []
    
    for test_case in test_cases:
        query = test_case["query"]
        expected = test_case["expected_answer"]
        
        # RAG-Antwort generieren
        answer, context = simulate_rag(query, documents)
        
        # Grundlegende Metriken berechnen
        # 1. Rouge-Score für Ähnlichkeit zur erwarteten Antwort
        rouge_scores = scorer.score(expected, answer)
        rouge_f1 = rouge_scores["rougeL"].fmeasure
        
        # 2. LLM-basierte Bewertung
        eval_prompt = f"""Bewerte die folgende Antwort auf die Frage: \"{query}\" 
        auf einer Skala von 1-10 für Genauigkeit, Vollständigkeit und Klarheit. 
        Erwartete Antwort: {expected}
        Tatsächliche Antwort: {answer}"""
        
        evaluation = evaluator_llm.invoke(eval_prompt)
        
        # Ergebnisse sammeln
        results.append({
            "query": query,
            "expected": expected,
            "answer": answer,
            "rouge_f1": rouge_f1,
            "llm_evaluation": evaluation.content
        })
    
    return results

# Evaluierung durchführen
evaluation_results = evaluate_rag_system(test_dataset, rag_documents)

# Ergebnisse anzeigen
for i, result in enumerate(evaluation_results):
    print(f"\nTest {i+1}: {result['query']}")
    print(f"Rouge-L F1: {result['rouge_f1']:.4f}")
    print(f"LLM-Bewertung: {result['llm_evaluation']}")

## 8. Zusammenfassung: Evaluierungsworkflow

Basierend auf den vorgestellten Methoden können wir einen umfassenden Evaluierungsworkflow für LLM- und RAG-Anwendungen erstellen.

In [ ]:
# Umfassender Evaluierungsworkflow für eine LLM-Anwendung
def comprehensive_evaluation_workflow(query, documents, reference_answer=None):
    print(f"Evaluierung für Anfrage: {query}")
    print("-" * 50)
    
    # 1. Datenschutz-Check und Anonymisierung
    safe_query, was_anonymized = prepare_safe_evaluation_data(query)
    if was_anonymized:
        print("⚠️ Anfrage enthielt personenbezogene Daten und wurde anonymisiert.")
        print(f"Anonymisierte Anfrage: {safe_query}")
    
    # 2. RAG-Antwort generieren
    answer, context = simulate_rag(safe_query, documents)
    print("\n📝 Generierte Antwort:")
    print(answer)
    
    # 3. Metriken berechnen
    metrics = {}
    
    # 3.1 Wenn eine Referenzantwort vorhanden ist, klassische NLP-Metriken verwenden
    if reference_answer:
        rouge_scores = scorer.score(reference_answer, answer)
        metrics["ROUGE-L F1"] = rouge_scores["rougeL"].fmeasure
        
        reference_tokens = nltk.word_tokenize(reference_answer.lower())
        answer_tokens = nltk.word_tokenize(answer.lower())
        metrics["BLEU"] = sentence_bleu([reference_tokens], answer_tokens)
    
    # 3.2 LLM-basierte Evaluierung für Qualität
    quality_eval = quality_chain.invoke({"query": safe_query, "answer": answer})
    
    # 3.3 RAG-spezifische Evaluierung
    faith_eval = faithfulness_chain.invoke({"context": context, "answer": answer})
    context_eval = context_eval_chain.invoke({"query": safe_query, "context": context})
    
    # 4. Ergebnisse zusammenfassen
    print("\n📊 Evaluierungsergebnisse:")
    if reference_answer:
        print(f"ROUGE-L F1: {metrics['ROUGE-L F1']:.4f}")
        print(f"BLEU Score: {metrics['BLEU']:.4f}")
    
    print("\n🔍 Qualitätsbewertung:")
    print(quality_eval["text"])
    
    print("\n✓ Treue zum Kontext:")
    print(faith_eval["text"])
    
    print("\n📚 Relevanz der abgerufenen Dokumente:")
    print(context_eval["text"])
    
    return {
        "query": query,
        "answer": answer,
        "context": context,
        "metrics": metrics,
        "quality_evaluation": quality_eval["text"],
        "faithfulness_evaluation": faith_eval["text"],
        "context_evaluation": context_eval["text"],
    }

# Beispiel für den umfassenden Evaluierungsworkflow
reference = "Für RAG-Systeme sind Chroma und Qdrant gut geeignete Vektordatenbanken. Chroma ist einfach zu nutzen und ideal für Prototyping, während Qdrant für Produktionsanwendungen mit hohen Anforderungen geeignet ist."
eval_results = comprehensive_evaluation_workflow(rag_query, rag_documents, reference_answer=reference)

## 9. Übungen

Hier sind einige Übungen, um Ihr Wissen über Evaluationsmethoden zu vertiefen:

### Übung 1: Eigenen Evaluator implementieren

Erstellen Sie einen benutzerdefinierten Evaluator für einen spezifischen Anwendungsfall (z.B. technische Dokumentation, medizinische Beratung, oder Kundensupport).

1. Definieren Sie 3-5 domänenspezifische Kriterien für Ihre Bewertung
2. Implementieren Sie diese mit dem `LabeledCriteriaEvaluator`
3. Testen Sie Ihren Evaluator mit mindestens 2 verschiedenen Antworten

### Übung 2: Vergleichende RAG-Evaluation

Vergleichen Sie zwei verschiedene RAG-Setups und evaluieren Sie ihre Leistung:

1. Setup A: Standard-RAG mit allen Dokumenten
2. Setup B: RAG mit Vorfilterung (z.B. nur die 2 relevantesten Dokumente verwenden)

Verwenden Sie die gleichen 3 Testanfragen für beide Setups und analysieren Sie die Unterschiede in den Ergebnissen.

### Übung 3: Dashboard für Evaluierungsergebnisse

Erstellen Sie ein einfaches Dashboard zur Visualisierung von Evaluierungsergebnissen:

1. Führen Sie mindestens 5 Tests mit verschiedenen Anfragen durch
2. Speichern Sie die Ergebnisse in einem DataFrame
3. Erstellen Sie Visualisierungen für verschiedene Metriken
4. Berechnen Sie Durchschnittswerte und identifizieren Sie Verbesserungspotenzial

## 10. Schlussfolgerungen und Best Practices

- **Kombinieren Sie mehrere Evaluierungsmethoden**: Nutzen Sie sowohl automatische Metriken als auch LLM-basierte Bewertungen für ein umfassenderes Bild.

- **Datenschutz beachten**: Stellen Sie sicher, dass Ihre Evaluierungsdaten keine personenbezogenen Informationen enthalten oder diese angemessen anonymisiert sind.

- **Kontinuierlich evaluieren**: Bauen Sie Evaluierung in Ihren Entwicklungsprozess ein, nicht nur als einmalige Aktivität am Ende.

- **Repräsentative Testdaten**: Stellen Sie sicher, dass Ihre Testfälle die tatsächlichen Nutzungsszenarien repräsentieren.

- **Menschliche Überprüfung**: Automatische Evaluierung sollte durch menschliche Überprüfung ergänzt werden, besonders bei wichtigen Anwendungsfällen.

- **Ergebnisse dokumentieren**: Halten Sie Ihre Evaluierungsergebnisse fest, um Verbesserungen im Laufe der Zeit zu verfolgen.